In [1]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import yaml

from src.models.ts2vec_src.ts2vec import TS2Vec
from src.data.preprocessing import read_data, preprocessing, data_to_np_tensor

%load_ext autoreload
%autoreload 2

/tmp/ipykernel_14354/469239892.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Preprocessing

In [3]:
# data = read_data('data/all_tickers.csv')
# train_data = preprocessing(
#     data, 
#     ['Open', 'High', 'Low', 'Close', 'Volume'],         
#     start_date = '2023-12-15',
#     end_date = '2023-12-21'
# )

# test_data = preprocessing(
#     data, 
#     ['Open', 'High', 'Low', 'Close', 'Volume'],          
#     start_date = '2023-12-21',
#     end_date = '2023-12-22',
#     tickers_save = train_data['Close'].columns
# )

In [4]:
# for feature in ['Open', 'High', 'Low', 'Close', 'Volume']:
#     print(train_data['Close'].shape[1], test_data['Close'].shape[1])

In [5]:
# train_ts = data_to_np_tensor(train_data)
# test_ts = data_to_np_tensor(test_data)

In [6]:
# model = TS2Vec(
#     input_dims=train_ts.shape[2],
#     device=1,
#     output_dims=320
# )

# loss_log = model.fit(
#     train_ts,
#     verbose=False
# )

# test_repr = model.encode(test_ts)
# #test_repr = model.encode(test_data, encoding_window='full_series')

In [7]:
# test_repr = model.encode(test_ts, encoding_window='full_series')
# print(test_repr.shape)
# test_repr

# mean stock price prediction

In [8]:
with open('configs/best_stocks_nans_rate.yaml') as f:
    best_stocks = yaml.load(f, Loader=yaml.FullLoader)

best_stocks = list(best_stocks.keys())
best_stocks

['AAPL',
 'ABBV',
 'ABT',
 'AMD',
 'BAC',
 'CMCSA',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'GOOG',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MRK',
 'MSFT',
 'NFLX',
 'NVDA',
 'PEP',
 'PFE',
 'PG',
 'TSLA',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [9]:
df = read_data('data/all_tickers.csv')
df_best = df.query("Stock in @best_stocks")
df_best

,Date,Time,Open,High,Low,Close,Volume,Datetime,Stock,Day_week
61,2024-01-30,22:58:00,188.130,188.17,188.110,188.14,1500,2024-01-30 22:58:00,AAPL,Tuesday
62,2024-01-30,22:57:00,188.220,188.22,188.080,188.11,5700,2024-01-30 22:57:00,AAPL,Tuesday
63,2024-01-30,22:56:00,188.220,188.27,188.215,188.22,2600,2024-01-30 22:56:00,AAPL,Tuesday
64,2024-01-30,22:55:00,188.150,188.20,188.140,188.20,2100,2024-01-30 22:55:00,AAPL,Tuesday
65,2024-01-30,22:54:00,188.180,188.18,188.140,188.18,2013,2024-01-30 22:54:00,AAPL,Tuesday
...,...,...,...,...,...,...,...,...,...,...
95882,2023-01-30,17:34:00,115.050,115.09,114.960,114.97,800,2023-01-30 17:34:00,XOM,Monday
95883,2023-01-30,17:33:00,115.090,115.27,114.910,115.15,4603,2023-01-30 17:33:00,XOM,Monday
95884,2023-01-30,17:32:00,115.250,115.25,115.000,115.03,850,2023-01-30 17:32:00,XOM,Monday
95885,2023-01-30,17:31:00,115.145,115.16,115.145,115.16,237,2023-01-30 17:31:00,XOM,Monday


In [10]:
df_best_h = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({
    'Open': 'first',
    'High': 'max',
    'Low': 'min',
    'Close': 'last',
    'Volume': "sum",
}).reset_index()

df_best_h

,Stock,Datetime,Open,High,Low,Close,Volume
0,AAPL,2023-01-30 17:00:00,145.15,145.53,144.18,145.13,291457
1,AAPL,2023-01-30 18:00:00,143.97,145.33,143.38,144.96,348830
2,AAPL,2023-01-30 19:00:00,144.07,144.30,143.50,143.95,215404
3,AAPL,2023-01-30 20:00:00,143.38,144.24,143.18,144.17,254970
4,AAPL,2023-01-30 21:00:00,143.31,143.81,143.01,143.38,192383
...,...,...,...,...,...,...,...
42191,XOM,2024-01-30 18:00:00,103.69,103.70,102.95,103.22,159389
42192,XOM,2024-01-30 19:00:00,103.79,103.96,103.49,103.67,90110
42193,XOM,2024-01-30 20:00:00,103.68,103.98,103.63,103.76,54009
42194,XOM,2024-01-30 21:00:00,104.15,104.16,103.67,103.68,61363


In [11]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'


train_data = preprocessing(
    df_best_h, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],         
    start_date = train_start,
    end_date = train_end,
    tickers_save = best_stocks
)

test_data = preprocessing(
    df_best_h, 
    ['Open', 'High', 'Low', 'Close', 'Volume'],          
    start_date = test_start,
    end_date = test_end,
    tickers_save = best_stocks
)

NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0
NAN in data  0


In [12]:
train_ts = data_to_np_tensor(train_data)
test_ts = data_to_np_tensor(test_data)

train_ts.shape, test_ts.shape

((28, 132, 5), (28, 24, 5))

In [13]:
model = TS2Vec(
    input_dims=train_ts.shape[2],
    device=2,
    output_dims=128
)

loss_log = model.fit(
    train_ts,
    verbose=False
)

In [14]:
train_repr = model.encode(train_ts)
test_repr = model.encode(test_ts)

train_repr.shape, test_repr.shape

((28, 132, 128), (28, 24, 128))

In [15]:
def stock_embeddigns_to_df(data_tensor: np.ndarray, stocks, dates) -> dict:    
    res = pd.DataFrame()
    for i, stock in enumerate(stocks):
        df = pd.DataFrame(data_tensor[i], index=dates) 
        df.columns = ['emb_' + str(i) for i in range(len(df.columns))]
        df['Stock'] = stock
        res = pd.concat([res, df])

    return res.reset_index(drop=False, names='Datetime')

X_train = stock_embeddigns_to_df(train_repr, stocks=train_data['Open'].columns, dates=train_data['Open'].index)
X_test = stock_embeddigns_to_df(test_repr, stocks=test_data['Open'].columns, dates=test_data['Open'].index)
X_train.head()

,Datetime,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_119,emb_120,emb_121,emb_122,emb_123,emb_124,emb_125,emb_126,emb_127,Stock
0,2023-10-02 17:00:00,2427.194824,-1261.177734,249.416061,-1326.052856,-1367.578125,-1558.728882,-4216.820801,-1760.594971,-3226.690430,...,-3736.978271,-2894.698486,59.054626,448.096893,-431.288330,-3278.026611,-3678.512451,-4118.010254,-865.218872,AAPL
1,2023-10-02 18:00:00,-4191.925781,-921.258484,-2921.642822,-4885.228027,-1271.211060,-6938.667480,-5882.696289,-1737.436646,-4292.594727,...,-2232.424316,-2460.291504,-3915.368652,-4717.567383,-1122.543701,-2810.496826,-1696.220703,-6516.091797,-51.443115,AAPL
2,2023-10-02 19:00:00,-1540.887329,-1819.440430,-2715.320801,-685.467407,-307.425812,-2639.099609,-1765.161377,-2044.926636,-3203.968262,...,-1362.843018,-1696.531250,-2086.554688,-1719.844971,-6393.981445,-4266.026367,-2607.339600,-2342.703857,-2697.456299,AAPL
3,2023-10-02 20:00:00,-1144.471313,-2759.480225,-2808.430176,665.400024,-1285.571777,-5994.809570,-1485.139160,-3820.880859,-2928.319092,...,-1212.408203,-1481.149048,-2768.698242,-2350.670654,-5365.892090,-3674.697998,-3923.934570,-2673.756348,-4784.291992,AAPL
4,2023-10-02 21:00:00,-64.300476,-925.543701,-2364.407227,-458.385864,-1492.317871,-3732.340820,-2279.485107,-3337.629150,-1244.936523,...,-1823.329834,-1594.620850,-2336.600586,49.904617,-5596.276367,-4646.608398,-1933.091797,-2176.987793,-2161.269531,AAPL


In [16]:
y = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
y.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [17]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]

In [18]:
y_name = 'Close'

df_train = pd.merge(X_train, y_train, on=['Stock', 'Datetime'])
df_test = pd.merge(X_test, y_test, on=['Stock', 'Datetime'])

X_train, y_train = df_train.drop(columns=[y_name, 'Datetime', 'Stock']), df_train[y_name]
X_test, y_test = df_test.drop(columns=[y_name, 'Datetime', 'Stock']), df_test[y_name]

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE

model = RandomForestRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

MAE(y_test, y_pred)

68.8434371264317

# baseline

In [20]:
X_baseline = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
X_baseline.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [21]:
n_shifts = 18
for i in range(1, n_shifts + 1):
    X_baseline[f'shift_{i}'] = X_baseline.groupby(by=['Stock']).shift(1)['Close']

X_baseline = X_baseline.dropna()
X_baseline = X_baseline.drop(columns='Close')
X_baseline.head()

,Stock,Datetime,shift_1,shift_2,shift_3,shift_4,shift_5,shift_6,shift_7,shift_8,shift_9,shift_10,shift_11,shift_12,shift_13,shift_14,shift_15,shift_16,shift_17,shift_18
1,AAPL,2023-01-30 18:00:00,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667,144.833667
2,AAPL,2023-01-30 19:00:00,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833,144.159833
3,AAPL,2023-01-30 20:00:00,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083,143.951083
4,AAPL,2023-01-30 21:00:00,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667,143.681667
5,AAPL,2023-01-30 22:00:00,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167,143.388167


In [22]:
y = df_best.set_index('Datetime').groupby(
    ['Stock', pd.Grouper( freq='h')],
).agg({'Close': 'mean'}).reset_index()
y.head()

,Stock,Datetime,Close
0,AAPL,2023-01-30 17:00:00,144.833667
1,AAPL,2023-01-30 18:00:00,144.159833
2,AAPL,2023-01-30 19:00:00,143.951083
3,AAPL,2023-01-30 20:00:00,143.681667
4,AAPL,2023-01-30 21:00:00,143.388167


In [23]:
train_start, train_end = '2023-10-01', '2023-11-01'
test_start, test_end = '2023-11-01', '2023-11-07'

y_train = y[(y['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(train_end).date())]

y_test = y[(y['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (y['Datetime'].dt.date < pd.Timestamp(test_end).date())]

X_train = X_baseline[(X_baseline['Datetime'].dt.date >= pd.Timestamp(train_start).date()) & 
             (X_baseline['Datetime'].dt.date < pd.Timestamp(train_end).date())]

X_test = X_baseline[(X_baseline['Datetime'].dt.date >= pd.Timestamp(test_start).date()) & 
             (X_baseline['Datetime'].dt.date < pd.Timestamp(test_end).date())]

In [24]:
y_name = 'Close'

df_train = pd.merge(X_train, y_train, on=['Stock', 'Datetime'])
df_test = pd.merge(X_test, y_test, on=['Stock', 'Datetime'])

X_train, y_train = df_train.drop(columns=[y_name, 'Datetime', 'Stock']), df_train[y_name]
X_test, y_test = df_test.drop(columns=[y_name, 'Datetime', 'Stock']), df_test[y_name]

In [25]:
X_train

,shift_1,shift_2,shift_3,shift_4,shift_5,shift_6,shift_7,shift_8,shift_9,shift_10,shift_11,shift_12,shift_13,shift_14,shift_15,shift_16,shift_17,shift_18
0,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695,171.311695
1,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000,173.344000
2,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167,173.639167
3,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500,173.219500
4,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333,173.224333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167,105.645167
3692,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667,105.780667
3693,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583,105.537583
3694,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186,105.716186


In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE

model = RandomForestRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

MAE(y_test, y_pred)

1.2767527237773275